In this hands-on we will use the `smolagents` library developed by HuggingFace. We use this library due to its simplicity, support for any LLM posted on HuggingFace Hub and its integration of Code Agents (more on that later).

<img src="https://camo.githubusercontent.com/c6efa99360afde7cf829dff3cad81e56573658c1843464dff1fbb30a8f63b082/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f68756767696e67666163652f646f63756d656e746174696f6e2d696d616765732f7265736f6c76652f6d61696e2f736d6f6c6167656e74732f736d6f6c6167656e74732e706e67" alt="drawing" width="400"/>

[Smolagents Documentation](https://huggingface.co/docs/smolagents/en/index)

In this hands-on we will:
- Understand why it's helpful to have agentic capabilities
- Understand how to use the `smolagents` library
- Understand the difference between a Tool Calling Agent and a Code Agent
- Implement a custom Agent leveraging the RAG pipeline that we implemented before

## 1. Setting up the environment

We will use `gpt-4o-mini` as our LLM for this hands-on. We could also use any model available on HuggingFace. 

In [ ]:
!pip install -r requirements.txt

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [25]:
from smolagents import LiteLLMModel
from IPython.display import display_markdown

model = LiteLLMModel(model_id="gpt-4o-mini")

In [33]:
def compute_agent_cost(agent, model_name="gpt-4o-mini"):
    if model_name == "gpt-4o-mini":
        input_token_price = 0.15 / 1000000
        output_token_price = 0.6 / 1000000
    elif model_name == "gpt-4o":
        input_token_price = 2.5 / 1000000
        output_token_price = 10 / 1000000        
    return (
        input_token_price * agent.monitor.get_total_token_counts()["input"]
        + output_token_price * agent.monitor.get_total_token_counts()["output"]
    )

## 2. Let's create our first Agent

## 2.1 Tool Calling Agent

In [5]:
from smolagents import ToolCallingAgent

# This is as simple as
agent = ToolCallingAgent(
    tools=[],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)

In [6]:
output = agent.run("What can you do?")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I can assist with various tasks including answering questions, providing information, generating text, solving mathematical operations, assisting with programming, and more. If you have a specific task in mind, please let me know!

Cost of the agent:  0.00016214999999999998


Ok, so our agent says that it can help us answering questions. Let's see how it goes.

In [7]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are recent news?"
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I cannot access external websites, including the one you provided. However, you can visit https://www.swissinfo.ch/eng/ directly to check the latest news.

Cost of the agent:  0.00016035


In [8]:
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [9]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you

I guess we forgot to give tools to our agent. Let's add a websearch tool. We can use the one provided by default by the smollagents library.

In [10]:
from smolagents import VisitWebpageTool

visit_webpage_tool = VisitWebpageTool()
agent = ToolCallingAgent(
    tools=[visit_webpage_tool],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name          ┃ Description                                  ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ visit_webpage │ Visits a webpage at the given url and reads  │ url (`string`): The url of the webpage to    │
    │               │ its content as a markdown string. Use this   │ visit.                                       │
    │               │ to browse webpages.                          │                                              │
    │ final_answer  │ Provides a final answer to the given         │ answer (`any`): The final answer to the      │
    │               │ problem.                                     │ problem                                      │
    └───────────────┴──────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [11]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are recent news?"
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

Recent news from SwissInfo includes:

1. **PostFinance cancels accounts of customers with Cuba links** - published on March 28, 2025.
2. **Frédéric Maire to leave Cinémathèque suisse** - published on March 28, 2025.
3. **Basel cannabis study reports improvements in smokers’ mental health** - published on March 28, 2025.
4. **Group reports rise in anti-Semitic incidents in French-speaking Switzerland** - published on March 28, 2025.
5. **Swiss court rejects 30km/h speed limit in Geneva** - published on March 28, 2025.
6. **ICRC pleads for release of German nurse held in Somalia for seven years** - published on March 28, 2025.

Cost of the agent:  0.002058


#### So now we have an agent that can answer questions using the LLM and also search the web for us.
#### Let's see what the agents does behind the scene.

In [12]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you

In [13]:
# Let's print the system prompt
print("""""", agent.system_prompt)

 You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to some tools.

The tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".
This Action/Observation can repeat N times, you should take several steps when needed.

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Action:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}

To provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "name": "final_answer

## Tools

A tool is an atomic function to be used by an agent. To be used by an LLM, it also needs a few attributes that constitute its API and will be used to describe to the LLM how to call this tool:

    A name
    A description
    Input types and descriptions
    An output type

The library provide a list of default tools: https://github.com/huggingface/smolagents/blob/28cfef22389a2830176b48be9fcc3e3d5793b87b/src/smolagents/default_tools.py#L102

- PythonInterpreterTool
- FinalAnswerTool
- UserInputTool
- DuckDuckGoSearchTool
- GoogleSearchTool
- VisitWebpageTool

With the `smolagents` library, there are two ways of declaring tool. Using the `@tool` decorator or using the `Tool` class.

The `@tool` decorator is a more concise way of declaring a tool, but it is less flexible than the `Tool` class.

###  Defining a Tool as a Python Class

In this class, we define:

- `name`: The tool’s name.
- `description`: A description used to populate the agent’s system prompt.
- `inputs`: A dictionary with keys type and description, providing information to help the Python interpreter process inputs.
- `output_type`: Specifies the expected output type.
- `forward`: The method containing the inference logic to execute.

In [14]:
from smolagents import Tool


class Sum(Tool):
    name = "sum"
    description = "This is a tool that can add two numbers. It returns the sum of the two numbers."
    inputs = {
        "number_1": {"type": "number", "description": "The first number to add."},
        "number_2": {"type": "number", "description": "The second number to add."},
    }
    output_type = "number"

    def forward(self, number_1: float, number_2: float) -> float:
        return number_1 + number_2

In [78]:
sum_tool = Sum()
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum          │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [79]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ sum 3 4                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum' with arguments: {'number_1': 3, 'number_2': 4}                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 7

[Step 1: Duration 0.72 seconds| Input tokens: 993 | Output tokens: 22]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '7'}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 7

[Step 2: Duration 0.70 seconds| Input tokens: 2,083 | Output tokens: 37]

7

Cost of the agent:  0.00033465


### The @tool Decorator

Using this approach, we define a function with:

- **A clear and descriptive function name** that helps the LLM understand its purpose.
- **Type hints for both inputs and outputs** to ensure proper usage.
- **A detailed description**, including an Args: section where each argument is explicitly described. These descriptions provide valuable context for the LLM, so it’s important to write them carefully.

In [18]:
from smolagents import tool


@tool
def sum_tool(number_1: float, number_2: float) -> float:
    """
    This is a tool that can add two numbers. It returns the sum of the two numbers.

    Args:
        number_1: The first number to add.
        number_2: The second number to add.
    """
    return number_1 + number_2

In [19]:
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [20]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ sum 3 4                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum_tool' with arguments: {'number_1': 3, 'number_2': 4}                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 7

[Step 1: Duration 0.55 seconds| Input tokens: 995 | Output tokens: 23]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '7'}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 7

[Step 2: Duration 0.61 seconds| Input tokens: 2,080 | Output tokens: 38]

7

Cost of the agent:  0.0003348


## Code Agent vs Tool Calling Agent

In [80]:
from datetime import date

today = date.today()
current_date = today.strftime("%Y-%m-%d")


In [81]:
from smolagents import DuckDuckGoSearchTool

web_search = DuckDuckGoSearchTool()
agent = ToolCallingAgent(
    model=model,
    tools=[sum_tool, web_search],
    verbosity_level=1,
    max_steps=10,
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum          │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
    │              │ your query (think a Google search) then       │ perform.                                     │
    │              │ returns the top search results.               │                                              │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [83]:
output = agent.run(
    f"You are an agent that can study financial market. Today's date is {current_date}. "+
    "What is the gain that Nvidia stock made in the last week?", reset=True
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent, model.model_id))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 2025-03-28. What is the gain that Nvidia      │
│ stock made in the last week?                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': 'Nvidia stock price March 21 2025'}                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: ## Search Results

|NVIDIA Corporation (NVDA) Stock Historical Prices & Data - Yahoo 
Finance](https://finance.yahoo.com/quote/NVDA/history/)
Discover historical prices for NVDA stock on Yahoo Finance. View daily, weekly or monthly format back to when 
NVIDIA Corporation stock was issued. ... Mar 21, 2025: 116.94: 117.99: 115.42: 117.70: ...

|Nvidia Stock Price Prediction 2025, 2026, 2027-2029](https://longforecast.com/nvidia)
Updated: 2025/03/27. Nvidia stock closed at 113.76 dollars the previous day. Nvidia Share Price Prediction For Next
Months and Years. Nvidia stock price forecast for March 2025. The forecast for beginning 125 dollars. Maximum price
125, minimum 98. Averaged Nvidia stock price for the month 114. At the end 107 dollars, change for March -14.4%.

|NVIDIA STOCK PRICE FORECAST TOMORROW, WEEK, 2025, 2026, 2027 - 30 Rates](https://30rates.com/nvidia)
Fed Funds Rate Forecast: 2025, 2026, 2027. Nvidia stock price forecast for March 2027. In the beginning at 317.22. 
Maximum 393.98, minimum 317.22. The averaged price 348.31. At the end of the month 364.80 dollars, change for March
15.0%. The forecast is updated on daily basis.

|NVIDIA (NVDA) Stock Chart and Price History 2025 - 
MarketBeat](https://www.marketbeat.com/stocks/NASDAQ/NVDA/chart/)
View NVIDIA (NASDAQ:NVDA) historical prices, past price performance, and an advanced NVDA stock chart at 
MarketBeat. ... NVDA Stock Chart for Thursday, March, 27, 2025. NVDA Chart by TradingView. Remove Ads. NVIDIA Stock
Price History ... 03/21/2025: $118.53: $117.70-.70%: $117.99: $115.42: 265.86 million shs: $2.87 trillion: 
03/20/2025: $117.52:

|NVIDIA - 26 Year Stock Price History | NVDA | 
MacroTrends](https://www.macrotrends.net/stocks/charts/NVDA/nvidia/stock-price-history)
Historical daily share price chart and data for NVIDIA since 1999 adjusted for splits and dividends. The latest 
closing stock price for NVIDIA as of March 25, 2025 is 120.69.. The all-time high NVIDIA stock closing price was 
149.43 on January 06, 2025.; The NVIDIA 52-week high stock price is 195.95, which is 62.4% above the current share 
price.; The NVIDIA 52-week low stock price is 75.61 ...

|Nvda Stock Price March 2025 | StatMuse Money](https://www.statmuse.com/money/ask/nvda-stock-price-march-2025)
The closing price for NVIDIA (NVDA) this month is $117.70, on Friday. It is down 4.7% for the month. The latest 
price is $117.70.

|NVIDIA (NVDA) Historical Stock Price Data - Stock Analysis](https://stockanalysis.com/stocks/nvda/history/)
A complete stock price history for NVIDIA Corporation (NVDA) stock, starting from its first trading day. Includes 
open, high, low, close and volume.

|Prediction: Nvidia Stock Is Going to Soar in 2025, Thanks to This 
...](https://www.fool.com/investing/2025/03/05/prediction-nvidia-stock-soar-2025-news-jensen-huan/)
Price as of March 21, 2025, 3:58 p.m. ET Nvidia ( NVDA -0.75% ) stock set a new record closing high just shy of 
$150 in January, but it has since declined by 16%.

|Where Will Nvidia Stock Be in 1 Year? | The Motley 
Fool](https://www.fool.com/investing/2025/03/19/where-will-nvidia-stock-be-in-1-year/)
Nvidia stock has been sliding throughout 2025. ... Price as of March 21, 2025, 3:58 p.m. ET. Nvidia stock has been 
sliding throughout 2025. It's been pretty hard to lose money investing in Nvidia ...

|NVIDIA (NVDA) Stock Forecast and Price Target 2025 - 
MarketBeat](https://www.marketbeat.com/stocks/NASDAQ/NVDA/forecast/)
11/21/2024: Citigroup Subscribe to MarketBeat All Access for the recommendation accuracy rating : ... March 23, 
2025. Please send any questions or comments about these NVIDIA pros and cons to contact@marketbeat.com. NVIDIA Bull
Case. ... The current stock price of NVidia is $117.70, which may present a buying opportunity for investors 
looking ...

[Step 1: Duration 2.41 seconds| Input tokens: 1,128 | Output tokens: 24]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The closing price of Nvidia stock on March 21, 2025,   │
│ was $118.53, and on March 27, 2025, it was $111.35. The gain over the week is calculated as follows: Gain =     │
│ (Price on March 27 - Price on March 21) / Price on March 21 * 100 = (111.35 - 118.53) / 118.53 * 100 = -6.9%.'} │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The closing price of Nvidia stock on March 21, 2025, was $118.53, and on March 27, 2025, it was 
$111.35. The gain over the week is calculated as follows: Gain = (Price on March 27 - Price on March 21) / Price on
March 21 * 100 = (111.35 - 118.53) / 118.53 * 100 = -6.9%.

[Step 3: Duration 5.15 seconds| Input tokens: 6,868 | Output tokens: 246]

The closing price of Nvidia stock on March 21, 2025, was $118.53, and on March 27, 2025, it was $111.35. The gain over the week is calculated as follows: Gain = (Price on March 27 - Price on March 21) / Price on March 21 * 100 = (111.35 - 118.53) / 118.53 * 100 = -6.9%.

Cost of the agent:  0.0011778


In [54]:
from smolagents import CodeAgent

agent = CodeAgent(
    model=model,
    add_base_tools=False,
    tools=[sum_tool, web_search],
    verbosity_level=1,
    max_steps=10,
)
agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
    │              │ your query (think a Google search) then       │ perform.                                     │
    │              │ returns the top search results.               │                                              │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [55]:
output = agent.run(
    f"You are an agent that can study financial market. Today's date is {current_date}. "+
    "What is the gain that Nvidia stock made in the last week?", reset=True
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent, model.model_id))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 2025-03-28. What is the gain that Nvidia      │
│ stock made in the last week?                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  nvidia_price_today_query = "Nvidia stock price March 28, 2025"                                                   
  nvidia_price_last_week_query = "Nvidia stock price March 21, 2025"                                               
                                                                                                                   
  price_today = web_search(query=nvidia_price_today_query)                                                         
  print("Nvidia stock price on March 28, 2025:", price_today)                                                      
                                                                                                                   
  price_last_week = web_search(query=nvidia_price_last_week_query)                                                 
  print("Nvidia stock price on March 21, 2025:", price_last_week)                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Nvidia stock price on March 28, 2025: ## Search Results

[Nvidia Stock Price Prediction 2025, 2026, 2027-2029](https://longforecast.com/nvidia)
Updated: 2025/03/27 Nvidia stock closed at 113.76 dollars the previous day. Nvidia Share Price Prediction For Next 
Months and Years Nvidia stock price forecast for March 2025. The forecast for beginning 125 dollars. Maximum price 
125, minimum 98. Averaged Nvidia stock price for the month 114. At the end 107 dollars, change for March -14.4%.

[NVIDIA STOCK PRICE FORECAST TOMORROW, WEEK, 2025, 2026, 2027 - 30 Rates](https://30rates.com/nvidia)
Nvidia stock price prediction for tomorrow, near days, this week and this month. Short term NVDA stock forecast 
updated TODAY! Nvidia share price prediction for 2025, 2026, 2027 in the table.

[Nvda Stock Price March 2025 | StatMuse Money](https://www.statmuse.com/money/ask/nvda-stock-price-march-2025)
The closing price for NVIDIA (NVDA) this month is $117.70, on Friday. It is down 4.7% for the month. The latest 
price is $117.70.

[Historical NVDA stock prices (quote) - NVIDIA Corporation](https://stockinvest.us/stock-price/NVDA)
Historical stock prices refer to a stock's recorded prices at various past points. These prices include several key
figures that help investors and analysts evaluate a stock's performance over time: : Open price for the trading 
day. : Highest price for the trading day. : Lowest price for the trading day. : Close price for the trading day. 
Additionally, historical prices often include: is ...

[Nvidia Stock Prices In 2025 | StatMuse Money](https://www.statmuse.com/money/ask/nvidia-stock-prices-in-2025)
The closing price for NVIDIA (NVDA) this year is $121.67, on Friday. It is down 10.5% for the year. The latest 
price is $121.67.

[NVIDIA (NVDA) Historical Stock Price Data](https://stockanalysis.com/stocks/nvda/history/)
A complete stock price history for NVIDIA Corporation (NVDA) stock, starting from its first trading day. Includes 
open, high, low, close and volume.

[NVIDIA (NVDA) Stock Chart and Price History 2025 - 
MarketBeat](https://www.marketbeat.com/stocks/NASDAQ/NVDA/chart/)
View NVIDIA (NASDAQ:NVDA) historical prices, past price performance, and an advanced NVDA stock chart at 
MarketBeat.

[NVIDIA (NVDA) Stock Forecast and Price Target 2025 - 
MarketBeat](https://www.marketbeat.com/stocks/NASDAQ/NVDA/forecast/)
NVDA's current price target is $171.51. Learn why top analysts are making this stock forecast for NVIDIA at 
MarketBeat.

[NVIDIA (NVDA) Price Prediction and Forecast 2025-2030 For March 
25](https://247wallst.com/forecasts/2025/03/25/nvidia-nvda-price-prediction-and-forecast/)
By the conclusion of 2030, 24/7 Wall Street estimates that NVIDIA's stock will be trading for $362.00, good for a 
202.90% increase over today's share price, based on an EPS of $7.24 and a P/E ...

[NVIDIA Corp (Nasdaq:NVDA) Historical Data | Historical Stock Price Data 
...](https://markets.financialcontent.com/stocks/quote/historical?Symbol=537:994529)
View 20 years of historical stock price data for NVIDIA Corp (Nasdaq:NVDA)
Nvidia stock price on March 21, 2025: ## Search Results

[NVIDIA Corporation (NVDA) Stock Historical Prices & Data - Yahoo 
Finance](https://finance.yahoo.com/quote/NVDA/history/)
Discover historical prices for NVDA stock on Yahoo Finance. View daily, weekly or monthly format back to when 
NVIDIA Corporation stock was issued. ... Mar 21, 2025: 116.94: 117.99: 115.42: 117.70: ...

[Historical NVDA stock prices (quote) - NVIDIA Corporation](https://stockinvest.us/stock-price/NVDA)
Range Low Price High Price Comment; 30 days: $104.77: $138.59: Wednesday, 26th Mar 2025 NVDA stock ended at 
$113.76.This is 5.74% less than the trading day before Tuesday, 25th Mar 2025. During the day the stock fluctuated 
5.44% from a day low at $112.71 to a day high of $118.84.: 90 days

[Nvidia Stock Price Prediction 2025, 2026, 2027-2029](https://longforecast.com/nvidia)
Updated: 2025/03/27. Nvidia stock 

[Step 1: Duration 6.27 seconds| Input tokens: 2,125 | Output tokens: 181]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  price_today = 117.70                                                                                             
  price_last_week = 118.53                                                                                         
                                                                                                                   
  gain_percentage = ((price_today - price_last_week) / price_last_week) * 100                                      
  final_answer(gain_percentage)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: -0.7002446637981931

[Step 2: Duration 3.03 seconds| Input tokens: 6,505 | Output tokens: 354]

Cost of the agent:  0.00118815


## Agent Hierarchy / MultiAgent

It is also possible to use a multi-agent system, where multiple agents can be used to solve a problem. This is useful when the problem is too complex for a single agent to solve. In this case, the agents can communicate with each other to solve the problem.

Another advantage is context size, as a single agent will store the full history of the steps, while a multi-agent system will store only the history of the steps of the agent that is currently active.

In [84]:
from smolagents import CodeAgent, DuckDuckGoSearchTool


web_agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
    add_base_tools=False,
    name="information_retriever_agent",
    description="An agent that can be called to run web search to obtain information. Call it as a function using the **task** argument.",
    verbosity_level=1,
)

manager_agent = CodeAgent(
    tools=[sum_tool],
    model=model,
    managed_agents=[web_agent],
    verbosity_level=1,
    description="An agent that manages other agent.",
    max_steps=10,
)

manager_agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ sum          │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
│   │              │ returns the sum of the two numbers.           │ add.                                         │
│   │              │                                               │ number_2 (`number`): The second number to    │
│   │              │                                               │ add.                                         │
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    └── information_retriever_agent | CodeAgent | gpt-4o-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: An agent that can be called to run web search to obtain information. Call it as a 
        │   function using the **task** argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name         ┃ Description                               ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search   │ Performs a duckduckgo web search based on │ query (`string`): The search query to    │
            │              │ your query (think a Google search) then   │ perform.                                 │
            │              │ returns the top search results.           │                                          │
            │ final_answer │ Provides a final answer to the given      │ answer (`any`): The final answer to the  │
            │              │ problem.                                  │ problem                                  │
            └──────────────┴───────────────────────────────────────────┴──────────────────────────────────────────┘

In [85]:
output = manager_agent.run(
    f"You are an agent that can study financial market. Today's date is {current_date}. "+
    "What is the gain that Nvidia stock made in the last week?", reset=True
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(manager_agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 2025-03-28. What is the gain that Nvidia      │
│ stock made in the last week?                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  information_retriever_agent(task="Retrieve Nvidia stock price on 2025-03-21 and 2025-03-28.")                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭───────────────────────────────────── New run - information_retriever_agent ─────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'information_retriever_agent'.                                                     │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Retrieve Nvidia stock price on 2025-03-21 and 2025-03-28.                                                       │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({"absolute_change": -3.94, "percentage_change": -3.35})                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'absolute_change': -3.94, 'percentage_change': -3.35}

[Step 3: Duration 2.85 seconds| Input tokens: 9,891 | Output tokens: 668]

Cost of the agent:  0.00188445


## Visualize Your Agent

In [ ]:
from smolagents import (
    GradioUI,
)

web_agent = CodeAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
    name="information_retriever_agent",
    description="An agent that can be called to run web search to obtain information. Call it as a function using the **task** argument.",
    verbosity_level=0,
)

manager_agent = CodeAgent(
    name="Manager Agent",
    tools=[],
    model=model,
    managed_agents=[web_agent],
    verbosity_level=0,
    description="An agent that manages other agent.",
    max_steps=10,
)

GradioUI(manager_agent).launch(
    share=True
)  # This is necessary to make it work in Renku but be careful

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://6bcbc0e6f0e7b7f2ed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# !!! Do not forget to stop the process of the previous cell before executing the next ones !!! 

## Another Multi Agent Example

We will create:

- an agent that can read a document and summarize it

- an agent that can search the web

- a manager agent that handle both previous agents

In [59]:
import requests
from src.data_processing import SimpleChunker, PDFExtractorAPI


@tool
def read_pdf_tool(pdf_file_path: str) -> str:
    """
    This tool reads a PDF file and returns the text content of the PDF file.

    Args:
        pdf_file_path: The path to the PDF file.
    """
    response, text, images = PDFExtractorAPI().extract_text_and_images(pdf_file_path)
    return text

In [60]:
tmp = read_pdf_tool("../data/hypoxy_stat_1page.pdf")
tmp

'Please cite this article in press as: Blume et al., HypoxyStat, a small-molecule form of hypoxia therapy that increases oxygen-hemoglobin affinity, Cell (2025), https://doi.org/10.1016/j.cell.2025.01.029\n\n![](_page_0_Picture_2.jpeg)\n\n### Article HypoxyStat, a small-molecule form of hypoxia therapy that increases oxygen-hemoglobin affinity\n\nSkyler Y. Blume,[1,](#page-0-0)[2](#page-0-1) Ankur Garg,[1,](#page-0-0)[2](#page-0-1) Yolanda Martı´-Mateos,[1,](#page-0-0)[2](#page-0-1) Ayush D. Midha,[1,](#page-0-0)[2](#page-0-1) Brandon T. L. Chew,[1,](#page-0-0)[2](#page-0-1) Baiwei Lin,[3](#page-0-2) Cecile Yu,[3](#page-0-2) Ryan Dick,[3](#page-0-2) Patrick S. Lee,[3](#page-0-2) Eva Situ,[3](#page-0-2) Richa Sarwaikar,[3](#page-0-2) Eric Green,[3](#page-0-2) Vyas Ramanan,[3](#page-0-2) Gijsbert Grotenbreg,[3](#page-0-2) Maarten Hoek,[3](#page-0-2) Christopher Sinz,[3](#page-0-2) and Isha H. Jain[1,](#page-0-0)[2,](#page-0-1)[4,](#page-0-3)[5,](#page-0-4)[\\\\*](#page-0-5)\n\n<span id="

In [61]:
agent_read_pdf = CodeAgent(
    name="read_pdf_agent",
    description="Reads and summarize a PDF file. Call it as a function using the **task** argument.",
    tools=[read_pdf_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

agent_web_search = CodeAgent(
    name="web_search_agent",
    description="Runs web searches for you. Call it as a function using the **task** argument.",
    tools=[DuckDuckGoSearchTool(), visit_webpage_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

agent = CodeAgent(
    name="medical_agent",
    tools=[],
    model=model,
    add_base_tools=False,
    managed_agents=[agent_read_pdf, agent_web_search],
    verbosity_level=1,
)

agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    ├── read_pdf_agent | CodeAgent | gpt-4o-mini
    │   ├── ✅ Authorized imports: []
    │   ├── 📝 Description: Reads and summarize a PDF file. Call it as a function using the **task** argument.
    │   └── 🛠️ Tools:
    │       ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    │       ┃ Name          ┃ Description                              ┃ Arguments                                ┃
    │       ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │       │ read_pdf_tool │ This tool reads a PDF file and returns   │ pdf_file_path (`string`): The path to    │
    │       │               │ the text content of the PDF file.        │ the PDF file.                            │
    │       │ web_search    │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
    │       │               │ on your query (think a Google search)    │ perform.                                 │
    │       │               │ then returns the top search results.     │                                          │
    │       │ visit_webpage │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
    │       │               │ reads its content as a markdown string.  │ to visit.                                │
    │       │               │ Use this to browse webpages.             │                                          │
    │       │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to the  │
    │       │               │ problem.                                 │ problem                                  │
    │       └───────────────┴──────────────────────────────────────────┴──────────────────────────────────────────┘
    └── web_search_agent | CodeAgent | gpt-4o-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Runs web searches for you. Call it as a function using the **task** argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description                              ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search    │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
            │               │ on your query (think a Google search)    │ perform.                                 │
            │               │ then returns the top search results.     │                                          │
            │ visit_webpage │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
            │               │ reads its content as a markdown string.  │ to visit.                                │
            │               │ Use this to browse webpages.             │                                          │
            │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to the  

In [62]:
output = agent.run(
    "Your task is the following: "+
    "Can you read the PDF file at '../data/hypoxy_stat_1page.pdf' and tell me what it is about? Also, can you give me the wikipedia definition of the area of research?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────── New run - medical_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ Your task is the following: Can you read the PDF file at '../data/hypoxy_stat_1page.pdf' and tell me what it is │
│ about? Also, can you give me the wikipedia definition of the area of research?                                  │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  pdf_summary = read_pdf_agent(task="Read and summarize the PDF file located at                                    
  '../data/hypoxy_stat_1page.pdf'.")                                                                               
  print(pdf_summary)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────────────────────────────────────── New run - read_pdf_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'read_pdf_agent'.                                                                  │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Read and summarize the PDF file located at '../data/hypoxy_stat_1page.pdf'.                                     │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  summary = {                                                                                                      
      "pdf_summary": {                                                                                             
          "title": "HypoxyStat Overview",                                                                          
          "description": "HypoxyStat is a small molecule that increases oxygen-hemoglobin binding, potentially     
  beneficial for hypoxia therapy in mitochondrial diseases. It has been shown to extend lifespan and improve       
  symptoms in animal models."                                                                                      
      },                                                                                                           
      "wikipedia_definition": {                                                                                    
          "title": "Hypoxia",                                                                                      
          "description": "Hypoxia is a condition of inadequate oxygen supply at the tissue level, which            
  significantly affects mitochondrial diseases by impairing energy production. It can be categorized as            
  generalized or local and is related to conditions that compromise cellular respiration."                         
      }                                                                                                            
  }                                                                                                                
                                                                                                                   
  final_answer(summary)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'pdf_summary': {'title': 'HypoxyStat Overview', 'description': 'HypoxyStat is a small molecule
that increases oxygen-hemoglobin binding, potentially beneficial for hypoxia therapy in mitochondrial diseases. It 
has been shown to extend lifespan and improve symptoms in animal models.'}, 'wikipedia_definition': {'title': 
'Hypoxia', 'description': 'Hypoxia is a condition of inadequate oxygen supply at the tissue level, which 
significantly affects mitochondrial diseases by impairing energy production. It can be categorized as generalized 
or local and is related to conditions that compromise cellular respiration.'}}

[Step 3: Duration 4.98 seconds| Input tokens: 24,551 | Output tokens: 1,106]

Cost of the agent:  0.00434625


## Exercise 1: Create your own RAG Agent

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/beating_gaia/classical_vs_agentic_rag.png)

First we will create a simple agent that can answer questions on a knowledge base, AKA a **RAG agent**.

1. Define a tool that retrieve documents from a knowledge base
2. Define an agent that uses the tool to retrieve documents and answer questions

In [63]:
from src.data_classes import Chunk
from src.data_processing import SimpleChunker, PDFExtractorAPI
from src.embedding import (
    OpenAITextEmbeddings,
    compute_openai_large_embedding_cost,
)
from src.vectorstore import (
    ChromaDBVectorStore,
    VectorStoreRetriever,
)

### Retriever Pipeline

In [64]:
data_folder = "../data"

pdf_files = [
    "Explainable_machine_learning_prediction_of_edema_a.pdf",
    "Modeling tumor size dynamics based on real‐world electronic health records.pdf",
]
example_pdf_file = "Explainable_machine_learning_prediction_of_edema_a.pdf"
example_pdf_path = os.path.join(data_folder, example_pdf_file)

vector_store_collection = "text_collection"

In [65]:
data_extractor = PDFExtractorAPI()
_, text, _ = data_extractor.extract_text_and_images(example_pdf_path)

In [66]:
file_metadata = {"source_text": example_pdf_file}

text_chunker = SimpleChunker(max_chunk_size=1000)

chunks = text_chunker.chunk_text(text, file_metadata)

In [67]:
embedding_model = OpenAITextEmbeddings()
embeddings = embedding_model.get_embedding([chunk.content for chunk in chunks])

In [68]:
vector_store = ChromaDBVectorStore(vector_store_collection)
vector_store.insert_documents(chunks, embeddings)

In [69]:
retriever = VectorStoreRetriever(embedding_model, vector_store)
results = retriever.retrieve("Who are the authors of the paper=", 5)
results

[[{'chunk_id': '48',
   'score': 1.265897274017334,
   'chunk': Chunk(chunk_id=48, content='performed the research. F.A. and N.T. analyzed the data. F.A., N.T., and R.C. contributed new reagents/analytical tools. ### **FUNDING INFORMATION** This study was sponsored by the healthcare business of Merck KGaA, Darmstadt, Germany (CrossRef Funder ID:10.13039/100009945). Editorial support was provided by Syneos Health, London, UK, and funded by the healthcare business of Merck KGaA, Darmstadt, Germany. ### **CONFLICT OF INTEREST STATEMENT** N.T. is an employee of Ares Trading S.A., Lausanne, Switzerland, an affiliate of Merck KGaA, Darmstadt, Germany. R.S., R.B. and A.J. are employees of the healthcare business of Merck KGaA, Darmstadt, Germany. K.B. was an employee of the healthcare business of Merck KGaA, Darmstadt, Germany at the time of the study. V.G. is an employee of Ares Trading S.A. Eysins, Switzerland, an affiliate of Merck KGaA, Darmstadt, Germany. F.A. and R.C.', metadata={'sourc

### Toolify

In [ ]:
# Create the retriever tool
@tool
def retriever_tool(query: str, number_of_chunks: int) -> list:
    ## Fill the docstring here
    
    return retriever.retrieve(query, number_of_chunks)

# Create the CodeAgent with the tool
rag_agent = CodeAgent(tools=[retriever_tool], model=model)
rag_agent.visualize()

In [70]:
# Create the retriever tool
@tool
def retriever_tool(query: str, number_of_chunks: int) -> list:
    """
    This is a tool that can search a document and extract the related information based on the query given. It returns a list of string
    Only use this tool if necessary to answer the query otherwise rely on your internal knowledge.
    
    Args:
        query: The user query
        number_of_chunks: number of chunks to return, by default it's 5.
    """
    
    return retriever.retrieve(query, number_of_chunks)

# Create the CodeAgent with the tool
rag_agent = CodeAgent(tools=[retriever_tool], model=model)
rag_agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name           ┃ Description                                  ┃ Arguments                                   ┃
    ┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ retriever_tool │ This is a tool that can search a document    │ query (`string`): The user query            │
    │                │ and extract the related information based on │ number_of_chunks (`integer`): number of     │
    │                │ the query given. It returns a list of string │ chunks to return, by default it's 5.        │
    │                │ Only use this tool if necessary to answer    │                                             │
    │                │ the query otherwise rely on your internal    │                                             │
    │                │ knowledge.                                   │                                             │
    │ final_answer   │ Provides a final answer to the given         │ answer (`any`): The final answer to the     │
    │                │ problem.                                     │ problem                                     │
    └────────────────┴──────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [71]:
# Call the agent
output = rag_agent.run("According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade 2+)?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade     │
│ 2+)?                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  results = retriever_tool(query="factors influencing higher-grade edema Grade 2+ SHAP analysis")                  
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'results = retriever_tool(query="factors influencing higher-grade edema Grade 2+ SHAP
analysis")' due to: TypeError: retriever_tool() missing 1 required positional argument: 'number_of_chunks'

[Step 1: Duration 3.85 seconds| Input tokens: 2,080 | Output tokens: 90]

...

[Step 2: Duration 2.92 seconds| Input tokens: 4,408 | Output tokens: 186]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  influential_factors = [                                                                                          
      "Past edema grade",                                                                                          
      "Albumin levels",                                                                                            
      "Age greater than 70 years",                                                                                 
      "Time until the next visit",                                                                                 
      "Lower cumulative doses"                                                                                     
  ]                                                                                                                
  final_answer(influential_factors)                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ['Past edema grade', 'Albumin levels', 'Age greater than 70 years', 'Time until the next 
visit', 'Lower cumulative doses']

[Step 3: Duration 9.47 seconds| Input tokens: 8,275 | Output tokens: 417]

In [74]:
output = rag_agent.run("What is the highest court of the USA?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the highest court of the USA?                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Supreme Court")                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Supreme Court

[Step 1: Duration 1.89 seconds| Input tokens: 2,066 | Output tokens: 70]

## Exercise 2: Make it a multi-agent system

Now that we built a RAG agent, we will improve it by adding a web search tool to it. This way, if the agent can't find the answer in the knowledge base, it will search the web for it.

However, we will transform our agent into a multi-agent system. This way, we will have one agent responsible for answering questions using the knowledge base and another agent responsible for searching the web.

1. Define a tool that searches the web
2. Define a new agent that uses the web search tool
3. Create a multi-agent system that uses both agents


In [75]:
#
from smolagents import DuckDuckGoSearchTool

web_search_agent = CodeAgent(
    name="web_search_agent",
    description="Runs web searches for you. Call it as a function using the **task** argument.",
    tools=[DuckDuckGoSearchTool(), visit_webpage_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

rag_agent = CodeAgent(name="medical_literature_agent",
                    description="Retrieve information from medical litterature. Call it as a function using the **task** argument.", 
                      tools=[retriever_tool], 
                      model=model)

agent = CodeAgent(
    tools=[],
    model=model,
    add_base_tools=False,
    managed_agents=[rag_agent, web_search_agent],
    verbosity_level=1,
)

In [76]:
# Call the agent
output = agent.run("According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade 2+)?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade     │
│ 2+)?                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  task = "Retrieve information from medical literature regarding SHAP analysis factors influencing higher-grade    
  edema (Grade 2+)."                                                                                               
  medical_literature_agent(task=task)                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭────────────────────────────────────── New run - medical_literature_agent ───────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'medical_literature_agent'.                                                        │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Retrieve information from medical literature regarding SHAP analysis factors influencing higher-grade edema     │
│ (Grade 2+).                                                                                                     │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 1: Duration 18.49 seconds| Input tokens: 4,014 | Output tokens: 542]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "Previous Edema Grade": "Persistence of previous edema grades indicates a higher likelihood of future Grade  
  2+ edema.",                                                                                                      
      "Albumin Levels": "Lower albumin levels are associated with an increased risk of Grade 2+ edema.",           
      "Age": "Individuals over 70 years are significantly more at risk.",                                          
      "Dosage Exposure": "Low doses before assessment can reduce the probability of developing Grade 2+ edema."    
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'Previous Edema Grade': 'Persistence of previous edema grades indicates a higher likelihood of
future Grade 2+ edema.', 'Albumin Levels': 'Lower albumin levels are associated with an increased risk of Grade 2+ 
edema.', 'Age': 'Individuals over 70 years are significantly more at risk.', 'Dosage Exposure': 'Low doses before 
assessment can reduce the probability of developing Grade 2+ edema.'}

[Step 2: Duration 5.09 seconds| Input tokens: 6,573 | Output tokens: 838]

In [77]:
output = agent.run("What is the current price of Nvidia stock?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the current price of Nvidia stock?                                                                      │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  nvidia_stock_price = web_search(task="current price of Nvidia stock")                                            
  print("Nvidia stock price:", nvidia_stock_price)                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'nvidia_stock_price = web_search(task="current price of Nvidia stock")' due to: 
InterpreterError: It is not permitted to evaluate other functions than the provided tools or functions 
defined/imported in previous code (tried to execute web_search).

[Step 1: Duration 1.67 seconds| Input tokens: 2,084 | Output tokens: 64]

...

[Step 2: Duration 9.72 seconds| Input tokens: 5,220 | Output tokens: 381]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The current price of Nvidia stock is approximately $120.69.")                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The current price of Nvidia stock is approximately $120.69.

[Step 3: Duration 1.66 seconds| Input tokens: 7,925 | Output tokens: 441]